# Task 2:  Instrument Segmentation Task (Testing)

In [1]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib import image
from numpy import savez_compressed
from torchvision import transforms
import copy
import random


import torch
from gan import Generator, Discriminator  # models are in gan.py file
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [2]:
def match_im_msk(img_files,msk_files):
    
    pure_img_names=[]
    pure_msk_names=[]

    for im in img_files:
        pure_img_names+= [os.path.basename(im)]
    for msk in msk_files:
        pure_msk_names+= [os.path.basename(msk)]
    
    return(pure_img_names,pure_msk_names)

In [3]:
# loading data
data_dir = "./data/task2/resized/"
img_files = glob.glob(data_dir+"images/*.jpg")
msk_files=glob.glob(data_dir+"masks/*.png")
pure_img_names,pure_msk_names=match_im_msk(img_files,msk_files)
pure_img_names==pure_msk_names

False

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std =[0.5, 0.5, 0.5],
    ),
])

In [5]:
loaded_images = list()
loaded_masks = list()

for i,filename in enumerate(img_files):
    # load image
    img_data = image.imread(filename)
    # store loaded image
    img_data_tr=transform(img_data.copy()).numpy()
    loaded_images.append(img_data_tr)
    if i%500 ==0:
        print('> loaded image %s %s' % (filename, img_data_tr.shape))
for i,filename in enumerate(msk_files):
    # load image
    msk_data = image.imread(filename)
    msk_data_tr=transform(msk_data.copy()).numpy()
    # store loaded image
    loaded_masks.append(msk_data_tr)
    if i%500 ==0:
        print('> loaded mask %s %s' % (filename, msk_data_tr.shape))


> loaded image ./data/task2/resized/images\ckcu8ty6z00003b5yzfaezbs5.jpg (3, 512, 512)
> loaded image ./data/task2/resized/images\ckd4gwbyy000r3b5ydyl6gfto.jpg (3, 512, 512)
> loaded mask ./data/task2/resized/masks\ckcu8ty6z00003b5yzfaezbs5.png (3, 512, 512)
> loaded mask ./data/task2/resized/masks\ckd4gwbyy000r3b5ydyl6gfto.png (3, 512, 512)


In [6]:
# we can show some samples
# s=np.random.randint(0,900)
# print(s)
# for j in range(s,s+1):
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     ax1.imshow(loaded_images[j])
#     ax2.imshow(loaded_masks[j])

## Instantiate a SegGAN Model

In [8]:
ngf= 64   # generator number of filters
ndf= 64   # discriminator number of filters
lrG=0.002 #0.0002
lrD=0.002
lamb=100  # loss weighting parameters
beta1=0.5 
beta2=0.999

In [9]:
print("number of samples",len(loaded_images))
print("image dim", loaded_images[0].shape)
print("mask  dim",loaded_masks[0].shape)

number of samples 590
image dim (3, 512, 512)
mask  dim (3, 512, 512)


In [10]:
G = Generator(3, ngf, 3)
D = Discriminator(6, ndf, 1)
G.normal_weight_init(mean=0.0, std=0.02)
D.normal_weight_init(mean=0.0, std=0.02)

print(D)


Discriminator(
  (conv1): ConvBlock(
    (conv): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBlock(
    (conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, trac

In [11]:
print(G)

Generator(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBlock(
    (conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_ru

In [ ]:
if not os.path.exists('id_log_task2.txt'):
    with open('id_log_task2.txt','w') as f:
        f.write('0')
with open('id_log_task2.txt','r') as f:
    run_id = int(f.read())
    run_id+=1 

with open('id_log_task2.txt','w') as f:
    f.write(str(run_id))

model_dir = 'GAN_model_task2/'
save_dir='GAN_results_task2/'
ex_dir=save_dir+str(run_id)+"/"


if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
if not os.path.exists(ex_dir):
    os.makedirs(ex_dir)
    
if not os.path.exists(model_dir):
    os.mkdir(model_dir)



In [13]:
# Loss functions
BCE_loss = torch.nn.BCELoss()#.cuda()
L1_loss = torch.nn.L1Loss()#.cuda()

# I used my laptop, cuda is an option
#G.cuda()
#D.cuda()


### GAN Training

In [15]:
# this is the last training trial, I used the whole dataset, but I sued 900:100 training:validation for parameters tuning

split_point=len(loaded_images)
X_train=np.array(loaded_images[:split_point])
Y_train=np.array(loaded_masks[:split_point])
im_train_names=np.array(pure_img_names[:split_point])
msk_train_names=np.array(pure_msk_names[:split_point])

# X_val=np.array(loaded_images[split_point:])
# Y_val=np.array(loaded_masks[split_point:])
# im_val_names=np.array(pure_img_names[split_point:])
# msk_val_names=np.array(pure_msk_names[split_point:])

print("train set:", len(Y_train),len(X_train))
# print("val   set:", len(Y_val),len(X_val))

train set: 590 590


In [16]:
class Seg(Dataset):
    def __init__(self,X,Y,im_names,mask_names):

        self.len=X.shape[0]
        self.xdata=torch.from_numpy(X)
        self.ydata=torch.from_numpy(Y)
        
        self.xnames=im_names
        self.ynames=mask_names
        
    def __getitem__(self,index):
        return (self.xdata[index],self.ydata[index],self.xnames[index],self.ynames[index])
    
    def __len__(self):
        return(self.len)


In [17]:
# A function to save sample during training
# I created the function to save features from the middle of the netork
# the purpouse was to use them in creating graph later, however I stick to the GANs in this phase

def save_epoch_pairs(imgs_batch,
                     masks_batch,
                     gen_masks_batch, 
                     im_fn_batch,
                     msk_fn_batch,
                     im_dir_name,
                     msk_dir_name,
                     gen_msk_dir_name,
                     feat_dir_name):
    
   
    for i in range(len(imgs_batch)):
        
        im_fn =im_fn_batch[i]
        msk_fn=msk_fn_batch[i]
        

        
        gen_mask=gen_masks_batch[i].detach().numpy()#.permute(1,2,0).detach().numpy().astype(np.uint8)

        save_genmsk_fn = gen_msk_dir_name + msk_fn
        gen_mask_ = (((gen_mask -gen_mask.min()) * 255) / (gen_mask.max() - gen_mask.min())).transpose(1, 2, 0).astype(np.uint8)
        msk_gen = Image.fromarray(gen_mask_)
        msk_gen.save(save_genmsk_fn)
        #plt.show()
        plt.close()
        
        #save_msk_fn = msk_dir_name + msk_fn
        #msk_real = Image.fromarray(target_msk)
       # msk_real.save(save_msk_fn)
        #plt.show()
        #plt.close()
        
        
        # Saving features....
#         save_feat_fn = feat_dir_name + msk_fn[:-3]+"npz"
#         feat = features_batch[i].detach().numpy()
#         savez_compressed(save_feat_fn, feat)
        #torch.save(feat, save_feat_fn)


In [18]:
batch_size=8

train_seg=Seg(X_train,Y_train,im_train_names,msk_train_names)
train_loader=DataLoader(dataset=train_seg,batch_size=batch_size,shuffle=True)

# val_seg=Seg(X_val,Y_val,im_val_names,msk_val_names)
# val_loader=DataLoader(dataset=val_seg,batch_size=2,shuffle=True)



In [18]:
# for i, (image_pix, mask_pix,im_nn,msk_nn) in enumerate(train_loader):
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     #ax1.imshow(image_pix[0])
#     #ax2.imshow(mask_pix[0])
#     print(im_nn[0])
#     print(msk_nn[0])
#     print()
    

In [19]:

# Optimizers
G_optimizer = torch.optim.Adam(G.parameters(), lr=lrG, betas=(beta1, beta2))
D_optimizer = torch.optim.Adam(D.parameters(), lr=lrD, betas=(beta1, beta2))

In [20]:
# Training GAN
D_avg_losses = []
G_avg_losses = []
num_epochs=10
step = 0
for epoch in range(num_epochs):
    
    epoch_dir=ex_dir+str(epoch)+"/"
    if not os.path.exists(epoch_dir):
        os.mkdir(epoch_dir)


    epoch_im_dir=epoch_dir+"images/"


    epoch_msk_dir=epoch_dir+"masks/"


    epoch_genmsk_dir=epoch_dir+"gen_masks/"
    if not os.path.exists(epoch_genmsk_dir):
        os.mkdir(epoch_genmsk_dir)


    epoch_feat_dir=epoch_dir+"features/"
    if not os.path.exists(epoch_feat_dir):
        os.mkdir(epoch_feat_dir)

    D_losses = []
    G_losses = []

    # training
    for i, (input, target, im_file_names, msk_file_names) in enumerate(train_loader):

        # input & target image data
        x_ = Variable(input)#.permute(0,3,1,2)#Variable(input.cuda())
        y_ = Variable(target)#.permute(0,3,1,2)#Variable(target.cuda())

        x_ = x_.float()#Variable(input.cuda())
        y_ = y_.float()#Variable(target.cuda())

        # Train discriminator with real data
        D_real_decision = D(x_, y_).squeeze()
        real_ = Variable(torch.ones(D_real_decision.size())) #Variable(torch.ones(D_real_decision.size()).cuda())
        D_real_loss = BCE_loss(D_real_decision, real_)

        # Train discriminator with fake data
        gen_image, _ = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        fake_ = Variable(torch.zeros(D_fake_decision.size()))#Variable(torch.zeros(D_fake_decision.size()).cuda())
        D_fake_loss = BCE_loss(D_fake_decision, fake_)

        # Back propagation
        D_loss = (D_real_loss + D_fake_loss) * 0.5
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
#         gen_image, features = G(x_)
        gen_image, _ = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        G_fake_loss = BCE_loss(D_fake_decision, real_)

        # L1 loss
        l1_loss = lamb * L1_loss(gen_image, y_)

        # Back propagation
        G_loss = G_fake_loss + l1_loss
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())
        
        #features_proj=F.interpolate(features,scale_factor=(2,2), mode='nearest')

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(train_loader), D_loss.item(), G_loss.item()))
        

#         if epoch>=0:
#             save_epoch_pairs(x_,
#                              y_,
#                              gen_image,
#                              features_proj,
#                              im_file_names,
#                              msk_file_names,
#                              epoch_im_dir,
#                              epoch_msk_dir,
#                              epoch_genmsk_dir,
#                              epoch_feat_dir)

        
        if epoch>=4:
            save_epoch_pairs(x_,
                             y_,
                             gen_image,
                             im_file_names,
                             msk_file_names,
                             epoch_im_dir,
                             epoch_msk_dir,
                             epoch_genmsk_dir,
                             epoch_feat_dir)

        
#         save_epoch_pairs(imgs_batch,
#                      masks_batch,
#                      gen_masks_batch, 
#                      im_fn_batch,
#                      msk_fn_batch,
#                      im_dir_name,
#                      msk_dir_name,
#                      gen_msk_dir_name):

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    # Show result for test image
    
#     val_input, val_target, val_img_names, val_msk_names = val_loader.__iter__().__next__()

#     gen_image = G(Variable(test_input))
#     gen_image = gen_image.cpu().data
#     plot_test_result(test_input, test_target, gen_image, epoch, save=True, ex_dir=ex_dir)
    
    save_model_fn=ex_dir+"Seg_GAN_task2.pth"
        
    torch.save({"g_state_dict":G.state_dict(),
                "d_state_dict":D.state_dict(),
                "g_opt_dict":G_optimizer.state_dict(),
                "d_opt_dict":G_optimizer.state_dict(),
                       },
                       save_model_fn
                      )

Epoch [1/12], Step [1/74], D_loss: 0.8164, G_loss: 104.6766
Epoch [1/12], Step [2/74], D_loss: 3.6631, G_loss: 87.7071
Epoch [1/12], Step [3/74], D_loss: 7.4452, G_loss: 66.7118
Epoch [1/12], Step [4/74], D_loss: 2.5863, G_loss: 50.7814
Epoch [1/12], Step [5/74], D_loss: 1.7079, G_loss: 36.9020
Epoch [1/12], Step [6/74], D_loss: 1.1948, G_loss: 25.1190
Epoch [1/12], Step [7/74], D_loss: 1.2417, G_loss: 23.3659
Epoch [1/12], Step [8/74], D_loss: 0.9004, G_loss: 34.9979
Epoch [1/12], Step [9/74], D_loss: 0.6892, G_loss: 16.1769
Epoch [1/12], Step [10/74], D_loss: 0.6801, G_loss: 13.4316
Epoch [1/12], Step [11/74], D_loss: 0.6520, G_loss: 17.6263
Epoch [1/12], Step [12/74], D_loss: 0.6990, G_loss: 22.5414
Epoch [1/12], Step [13/74], D_loss: 0.9663, G_loss: 18.5735
Epoch [1/12], Step [14/74], D_loss: 0.7712, G_loss: 23.5054
Epoch [1/12], Step [15/74], D_loss: 0.7646, G_loss: 21.2442
Epoch [1/12], Step [16/74], D_loss: 0.6975, G_loss: 10.5546
Epoch [1/12], Step [17/74], D_loss: 0.5772, G_lo

Epoch [2/12], Step [64/74], D_loss: 3.6273, G_loss: 22.4163
Epoch [2/12], Step [65/74], D_loss: 1.4541, G_loss: 19.5868
Epoch [2/12], Step [66/74], D_loss: 0.5142, G_loss: 22.5306
Epoch [2/12], Step [67/74], D_loss: 0.6417, G_loss: 22.3412
Epoch [2/12], Step [68/74], D_loss: 0.3395, G_loss: 23.6242
Epoch [2/12], Step [69/74], D_loss: 0.0951, G_loss: 22.2257
Epoch [2/12], Step [70/74], D_loss: 0.0840, G_loss: 19.6008
Epoch [2/12], Step [71/74], D_loss: 0.0491, G_loss: 22.9954
Epoch [2/12], Step [72/74], D_loss: 0.0509, G_loss: 16.1982
Epoch [2/12], Step [73/74], D_loss: 0.0444, G_loss: 22.6112
Epoch [2/12], Step [74/74], D_loss: 0.0821, G_loss: 21.9692
Epoch [3/12], Step [1/74], D_loss: 0.0976, G_loss: 27.1448
Epoch [3/12], Step [2/74], D_loss: 0.0728, G_loss: 22.9230
Epoch [3/12], Step [3/74], D_loss: 0.0777, G_loss: 26.4270
Epoch [3/12], Step [4/74], D_loss: 0.0539, G_loss: 19.5949
Epoch [3/12], Step [5/74], D_loss: 0.0335, G_loss: 20.6488
Epoch [3/12], Step [6/74], D_loss: 0.0337, G_

Epoch [4/12], Step [54/74], D_loss: 0.7715, G_loss: 8.6341
Epoch [4/12], Step [55/74], D_loss: 0.6750, G_loss: 11.3408
Epoch [4/12], Step [56/74], D_loss: 0.6674, G_loss: 12.5870
Epoch [4/12], Step [57/74], D_loss: 0.8071, G_loss: 10.9132
Epoch [4/12], Step [58/74], D_loss: 0.6026, G_loss: 8.0134
Epoch [4/12], Step [59/74], D_loss: 0.6569, G_loss: 8.3448
Epoch [4/12], Step [60/74], D_loss: 0.6902, G_loss: 8.2494
Epoch [4/12], Step [61/74], D_loss: 1.0621, G_loss: 7.3979
Epoch [4/12], Step [62/74], D_loss: 0.4948, G_loss: 16.0555
Epoch [4/12], Step [63/74], D_loss: 0.7669, G_loss: 9.7748
Epoch [4/12], Step [64/74], D_loss: 0.6799, G_loss: 17.0471
Epoch [4/12], Step [65/74], D_loss: 0.7125, G_loss: 9.2283
Epoch [4/12], Step [66/74], D_loss: 0.7376, G_loss: 18.9375
Epoch [4/12], Step [67/74], D_loss: 0.6544, G_loss: 17.5404
Epoch [4/12], Step [68/74], D_loss: 0.6991, G_loss: 8.9946
Epoch [4/12], Step [69/74], D_loss: 0.4979, G_loss: 14.2648
Epoch [4/12], Step [70/74], D_loss: 0.4605, G_lo

Epoch [6/12], Step [45/74], D_loss: 0.8788, G_loss: 16.1560
Epoch [6/12], Step [46/74], D_loss: 0.6009, G_loss: 6.7689
Epoch [6/12], Step [47/74], D_loss: 0.7228, G_loss: 7.5266
Epoch [6/12], Step [48/74], D_loss: 0.4761, G_loss: 11.5514
Epoch [6/12], Step [49/74], D_loss: 0.5714, G_loss: 7.7961
Epoch [6/12], Step [50/74], D_loss: 0.5994, G_loss: 14.0744
Epoch [6/12], Step [51/74], D_loss: 0.8918, G_loss: 4.4935
Epoch [6/12], Step [52/74], D_loss: 0.7048, G_loss: 12.2766
Epoch [6/12], Step [53/74], D_loss: 0.6400, G_loss: 11.0051
Epoch [6/12], Step [54/74], D_loss: 0.9147, G_loss: 8.1246
Epoch [6/12], Step [55/74], D_loss: 0.6672, G_loss: 7.9979
Epoch [6/12], Step [56/74], D_loss: 0.7966, G_loss: 5.6790
Epoch [6/12], Step [57/74], D_loss: 0.5949, G_loss: 15.5744
Epoch [6/12], Step [58/74], D_loss: 0.7919, G_loss: 12.9706
Epoch [6/12], Step [59/74], D_loss: 1.1981, G_loss: 5.5482
Epoch [6/12], Step [60/74], D_loss: 0.7021, G_loss: 7.2868
Epoch [6/12], Step [61/74], D_loss: 0.7570, G_los

Epoch [8/12], Step [36/74], D_loss: 0.6878, G_loss: 6.0949
Epoch [8/12], Step [37/74], D_loss: 1.0966, G_loss: 8.0461
Epoch [8/12], Step [38/74], D_loss: 0.5147, G_loss: 6.6833
Epoch [8/12], Step [39/74], D_loss: 0.6271, G_loss: 8.3480
Epoch [8/12], Step [40/74], D_loss: 0.8736, G_loss: 12.9121
Epoch [8/12], Step [41/74], D_loss: 1.0000, G_loss: 17.4640
Epoch [8/12], Step [42/74], D_loss: 0.6725, G_loss: 10.1333
Epoch [8/12], Step [43/74], D_loss: 0.7200, G_loss: 3.8886
Epoch [8/12], Step [44/74], D_loss: 0.7295, G_loss: 9.7493
Epoch [8/12], Step [45/74], D_loss: 0.5711, G_loss: 9.2779
Epoch [8/12], Step [46/74], D_loss: 0.3516, G_loss: 15.7249
Epoch [8/12], Step [47/74], D_loss: 0.5459, G_loss: 8.5758
Epoch [8/12], Step [48/74], D_loss: 0.9416, G_loss: 4.1107
Epoch [8/12], Step [49/74], D_loss: 1.1392, G_loss: 8.8272
Epoch [8/12], Step [50/74], D_loss: 0.9394, G_loss: 10.1726
Epoch [8/12], Step [51/74], D_loss: 1.1061, G_loss: 6.6870
Epoch [8/12], Step [52/74], D_loss: 0.6304, G_loss:

Epoch [10/12], Step [27/74], D_loss: 0.6183, G_loss: 4.4356
Epoch [10/12], Step [28/74], D_loss: 0.7104, G_loss: 5.7274
Epoch [10/12], Step [29/74], D_loss: 0.6678, G_loss: 12.2581
Epoch [10/12], Step [30/74], D_loss: 0.5930, G_loss: 5.6117
Epoch [10/12], Step [31/74], D_loss: 0.8921, G_loss: 5.6545
Epoch [10/12], Step [32/74], D_loss: 0.7874, G_loss: 6.6932
Epoch [10/12], Step [33/74], D_loss: 0.7563, G_loss: 7.2058
Epoch [10/12], Step [34/74], D_loss: 0.7804, G_loss: 7.7556
Epoch [10/12], Step [35/74], D_loss: 0.6069, G_loss: 7.7265
Epoch [10/12], Step [36/74], D_loss: 0.4801, G_loss: 9.7121
Epoch [10/12], Step [37/74], D_loss: 0.8368, G_loss: 4.7090
Epoch [10/12], Step [38/74], D_loss: 0.6917, G_loss: 5.6494
Epoch [10/12], Step [39/74], D_loss: 0.5479, G_loss: 9.2519
Epoch [10/12], Step [40/74], D_loss: 0.4741, G_loss: 7.2720
Epoch [10/12], Step [41/74], D_loss: 0.7977, G_loss: 17.9827
Epoch [10/12], Step [42/74], D_loss: 0.6317, G_loss: 12.1301
Epoch [10/12], Step [43/74], D_loss: 

Epoch [12/12], Step [16/74], D_loss: 0.5773, G_loss: 8.4710
Epoch [12/12], Step [17/74], D_loss: 0.8332, G_loss: 9.0241
Epoch [12/12], Step [18/74], D_loss: 0.8402, G_loss: 4.3470
Epoch [12/12], Step [19/74], D_loss: 0.5848, G_loss: 6.6491
Epoch [12/12], Step [20/74], D_loss: 0.6901, G_loss: 4.3593
Epoch [12/12], Step [21/74], D_loss: 0.5334, G_loss: 7.0920
Epoch [12/12], Step [22/74], D_loss: 0.5670, G_loss: 3.7653
Epoch [12/12], Step [23/74], D_loss: 0.5192, G_loss: 7.6613
Epoch [12/12], Step [24/74], D_loss: 0.4774, G_loss: 9.5605
Epoch [12/12], Step [25/74], D_loss: 0.5175, G_loss: 6.2934
Epoch [12/12], Step [26/74], D_loss: 0.5838, G_loss: 6.2299
Epoch [12/12], Step [27/74], D_loss: 0.3385, G_loss: 11.3816
Epoch [12/12], Step [28/74], D_loss: 0.8856, G_loss: 6.7797
Epoch [12/12], Step [29/74], D_loss: 0.8156, G_loss: 5.3041
Epoch [12/12], Step [30/74], D_loss: 0.6974, G_loss: 7.4900
Epoch [12/12], Step [31/74], D_loss: 0.7089, G_loss: 4.5035
Epoch [12/12], Step [32/74], D_loss: 0.